In [1]:
!pip install psycopg

ERROR: Could not find a version that satisfies the requirement psycopg (from versions: none)
ERROR: No matching distribution found for psycopg


In [2]:
!pip install imblearn

In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import seaborn as sns

In [4]:
#Columns to use according to ERD
columns = ['Source','Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Sunrise_Sunset', 'Start_Lng','Side', 'City', 'County',
          'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)', 'Weather_Condition']

#Columns to use according to ERD
#columns = ['ID', 'Severity', 'Start_Time', 'End_Time', 'State', 'Country', 'Visibility(mi)', 'Weather_Condition', 'Sunrise_Sunset', 'Civil_Twilight']

In [5]:
#Reading csv data into df
df = pd.read_csv('df_new_100.csv', usecols = columns)
df.head()

,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Side,City,County,Visibility(mi),Wind_Direction,Wind_Speed(mph),Weather_Condition,Sunrise_Sunset
0,MapQuest,3,2020-02-17 21:04:02,2020-02-17 22:21:33,29.848141,-95.333038,L,Houston,Harris,8.0,SSE,12.0,Mostly Cloudy,Night
1,Bing,2,2018-09-07 14:37:59,2018-09-07 15:52:21,34.072740,-117.783240,R,Pomona,Los Angeles,10.0,WSW,9.2,Clear,Day
2,MapQuest,2,2019-12-06 16:13:06,2019-12-06 17:27:45,35.496880,-97.619003,R,Oklahoma City,Oklahoma,10.0,N,13.0,Fair,Day
3,MapQuest,2,2020-06-27 04:31:07,2020-06-27 05:35:32,37.535553,-122.074997,R,Fremont,Alameda,10.0,N,9.0,Cloudy,Night
4,Bing,1,2020-06-01 07:28:32,2020-06-01 07:43:32,40.046390,-76.334290,R,Lancaster,Lancaster,10.0,W,8.0,Fair,Day


In [6]:
#Drop null values, subject to change
df.dropna(axis=0,inplace=True)
df.isnull().sum().sum()

0

In [14]:
#Labelling output and input, subject to change
y = df['Severity']
X = df.drop(['Severity', 'Start_Time', 'End_Time', 'Visibility(mi)', 'Start_Lat', 'Start_Lng', 'Source', 'Side'],axis=1)

In [15]:
X.shape

(100000, 6)

In [16]:
#Use dummy variables for input features, convert to numerical data 
X = pd.get_dummies(X, columns=['Weather_Condition', 'Sunrise_Sunset', 'City', 'County', 'Wind_Direction'])
X.head

<bound method NDFrame.head of        Wind_Speed(mph)  Weather_Condition_Blowing Dust  \
0                 12.0                               0   
1                  9.2                               0   
2                 13.0                               0   
3                  9.0                               0   
4                  8.0                               0   
...                ...                             ...   
99995             17.3                               0   
99996              3.5                               0   
99997             13.0                               0   
99998              3.0                               0   
99999              6.9                               0   

       Weather_Condition_Blowing Dust / Windy  Weather_Condition_Blowing Snow  \
0                                           0                               0   
1                                           0                               0   
2                             

In [17]:
# Correlation plot
#plt.figure(figsize=(15,8))
#corr = df.corr()
#ax = sns.heatmap(
    #corr, 
    #vmin=-1, vmax=1, center=0,
    #cmap=sns.diverging_palette(20, 220, n=200),
    #square=True
#)
#plt.title("Correlation Plot")
#ax.set_xticklabels(
    #ax.get_xticklabels(),
    #rotation=45,
    #horizontalalignment='right'
#);

In [18]:
#Balancing the dataset
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

[(1, 67974), (2, 67974), (3, 67974), (4, 67974)]


In [19]:
#splitting the data into training and test datasets 
X_train,X_test,y_train,y_test = train_test_split(X_resampled,y_resampled,test_size=0.3, random_state=42)

In [20]:
#Decision Tree Classifier model on training data and predictions
model = DecisionTreeClassifier()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [21]:
# Accuracy 
accuracy = accuracy_score(y_test,y_pred)
accuracy

0.8829825056087485

In [22]:
#Classification report
from sklearn.metrics import classification_report
report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           1       0.97      1.00      0.99     20398
           2       0.82      0.71      0.76     20233
           3       0.78      0.82      0.80     20407
           4       0.95      1.00      0.97     20531

    accuracy                           0.88     81569
   macro avg       0.88      0.88      0.88     81569
weighted avg       0.88      0.88      0.88     81569



In [23]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict_proba(X_test)
y_pred

array([[0.00652765, 0.55484604, 0.32621912, 0.11240719],
       [0.78457566, 0.1087348 , 0.03286351, 0.07382602],
       [0.90117371, 0.05769631, 0.01793997, 0.02319001],
       ...,
       [0.55602618, 0.3970708 , 0.03767965, 0.00922337],
       [0.00124898, 0.33153148, 0.34514591, 0.32207363],
       [0.08367676, 0.17438203, 0.11242057, 0.62952064]])